# Time Series Prediction Analysis

This notebook demonstrates time series forecasting using LSTM and Prophet models with comprehensive feature engineering and hyperparameter tuning.

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data.data_loader import DataLoader
from features.feature_engineer import FeatureEngineer
from models.lstm_model import LSTMPredictor
from models.prophet_model import ProphetPredictor
from evaluation.evaluator import ModelEvaluator
from utils.config import Config

plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Data Loading and Exploration

In [ ]:
# Load stock data
config = Config('stock')
data_loader = DataLoader(config)
raw_data = data_loader.load_data()

print(f"Data shape: {raw_data.shape}")
print(f"Date range: {raw_data['Date'].min()} to {raw_data['Date'].max()}")
raw_data.head()

In [ ]:
# Visualize raw data
plt.figure(figsize=(12, 6))
plt.plot(raw_data['Date'], raw_data['Close'])
plt.title('Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Feature Engineering

In [ ]:
# Create features
feature_engineer = FeatureEngineer(config)
processed_data = feature_engineer.create_features(raw_data)

print(f"Number of features: {len(processed_data['feature_names'])}")
print(f"Training samples: {len(processed_data['X_train'])}")
print(f"Test samples: {len(processed_data['X_test'])}")
print(f"LSTM sequences - Train: {len(processed_data['X_train_seq'])}, Test: {len(processed_data['X_test_seq'])}")

## 3. Model Training and Prediction

In [ ]:
# Train LSTM model
lstm_model = LSTMPredictor(config)
lstm_predictions = lstm_model.train_and_predict(processed_data)

print(f"LSTM predictions shape: {lstm_predictions.shape}")

In [ ]:
# Train Prophet model
prophet_model = ProphetPredictor(config)
prophet_predictions = prophet_model.train_and_predict(processed_data)

print(f"Prophet predictions shape: {prophet_predictions.shape}")

## 4. Model Evaluation

In [ ]:
# Evaluate models
evaluator = ModelEvaluator(config)

lstm_metrics = evaluator.evaluate(lstm_predictions, processed_data['y_test_seq'])
prophet_metrics = evaluator.evaluate(prophet_predictions, processed_data['y_test'])

print("LSTM Metrics:")
for metric, value in lstm_metrics.items():
    print(f"  {metric}: {value:.4f}")

print("\nProphet Metrics:")
for metric, value in prophet_metrics.items():
    print(f"  {metric}: {value:.4f}")

## 5. Visualization

In [ ]:
# Plot predictions
plt.figure(figsize=(15, 8))

# Plot actual values
plt.plot(processed_data['y_test'][:len(prophet_predictions)], label='Actual', color='black', linewidth=2)

# Plot predictions
if len(lstm_predictions) > 0:
    plt.plot(lstm_predictions, label='LSTM', color='blue', alpha=0.7)

if len(prophet_predictions) > 0:
    plt.plot(prophet_predictions, label='Prophet', color='red', alpha=0.7)

plt.title('Time Series Predictions Comparison')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()